In [3]:
import random
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from torchsummary import summary
from thop import profile

# Initialize random seed for reproducibility
seed = 1787
random.seed(seed)
np.random.seed(seed)
th.manual_seed(seed)
th.cuda.manual_seed(seed)
th.cuda.manual_seed_all(seed)
th.backends.cudnn.deterministic = True
th.backends.cudnn.benchmark = False

# Set device
device = th.device("cuda" if th.cuda.is_available() else "cpu")

custom_epochs = 1
AP_custom_epochs = 1

prune_value=[1,2,4]
#prune_limits=[6]*18*3
prune_limits=[8]*18 + [15]*18 + [30]*18 #Make it 9, 9, 9, figure out what these 2 actually mean
#8, 17, 34s

optim_lr = 0.1
lamda = 0.01
alpha = 0.0001
beta = 0.0001

AP_alpha = 0.0001

regularization_prune_percentage = 0.02
decorrelation_lower_bound = 0.3
decorrelation_higher_bound = 0.4

trainloader = th.utils.data.DataLoader(
    torchvision.datasets.CIFAR10('../data', download=True, train=True,
                               transform=transforms.Compose([transforms.ToTensor()])),
    batch_size=100, shuffle=True)

testloader = th.utils.data.DataLoader(
    torchvision.datasets.CIFAR10('../data', download=True, train=False,
                               transform=transforms.Compose([transforms.ToTensor()])),
    batch_size=100, shuffle=True)

class Network():

    def weight_init(self, m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            if self.a_type == 'relu':
                init.kaiming_normal_(m.weight.data, nonlinearity=self.a_type)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'leaky_relu':
                init.kaiming_normal_(m.weight.data, nonlinearity=self.a_type)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'tanh':
                g = init.calculate_gain(self.a_type)
                init.xavier_uniform_(m.weight.data, gain=g)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'sigmoid':
                g = init.calculate_gain(self.a_type)
                init.xavier_uniform_(m.weight.data, gain=g)
                init.constant_(m.bias.data, 0)
            else:
                raise
                return NotImplemented


    def one_hot(self, y, gpu):

        try:
            y = th.from_numpy(y)
        except TypeError:
            None

        y_1d = y
        if gpu:
            y_hot = th.zeros((y.size(0), th.max(y).int()+1)).cuda()
        else:
            y_hot = th.zeros((y.size(0), th.max(y).int()+1))

        for i in range(y.size(0)):
            y_hot[i, y_1d[i].int()] = 1

        return y_hot

   
    def best_tetr_acc(self, prunes):
        print("prunes values id ", prunes)
        tr_acc = self.train_accuracy[prunes:]
        te_acc = self.test_accuracy[prunes:]
        best_te_acc = max(te_acc)
        indices = [i for i, x in enumerate(te_acc) if x == best_te_acc]
        temp_tr_acc = []
        for i in indices:
            temp_tr_acc.append(tr_acc[i])
        best_tr_acc = max(temp_tr_acc)

        del self.test_accuracy[prunes:]
        del self.train_accuracy[prunes:]
        self.test_accuracy.append(best_te_acc)
        self.train_accuracy.append(best_tr_acc)
        return best_te_acc, best_tr_acc

    def best_tetr_acc(self):
        tr_acc = self.train_accuracy[:]
        te_acc = self.test_accuracy[:]
        best_te_acc = max(te_acc)
        indices = [i for i, x in enumerate(te_acc) if x == best_te_acc]
        temp_tr_acc = []
        for i in indices:
            temp_tr_acc.append(tr_acc[i])
        best_tr_acc = max(temp_tr_acc)

        del self.test_accuracy[prunes:]
        del self.train_accuracy[prunes:]
        self.test_accuracy.append(best_te_acc)
        self.train_accuracy.append(best_tr_acc)
        return best_te_acc, best_tr_acc

    def create_folders(self, total_convs):
        main_dir = strftime("/Results/%b%d_%H:%M:%S%p", localtime()) + "_resnet_110/"
        import os
        current_dir = os.path.abspath(os.path.dirname(__file__))
        par_dir = os.path.abspath(current_dir + "/../")
        parent_dir = par_dir + main_dir
        path2 = os.path.join(parent_dir, "layer_file_info")
        os.makedirs(path2)
        return parent_dir

    def get_writerow(self, k):
        s = 'wr.writerow(['

        for i in range(k):
            s = s + 'd[' + str(i) + ']'
            if i < k - 1:
                s = s + ','
            else:
                s = s + '])'

        return s


    def get_logger(self,file_path):

        logger = logging.getLogger('gal')
        log_format = '%(asctime)s | %(message)s'
        formatter = logging.Formatter(log_format, datefmt='%m/%d %I:%M:%S %p')
        file_handler = logging.FileHandler(file_path)
        file_handler.setFormatter(formatter)
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(formatter)

        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)
        logger.setLevel(logging.INFO)

        return logger

class PruningMethod:
    
    def prune_filters(self, indices):
        conv_layer = 0

        for layer_name, layer_module in self.named_modules():
            if isinstance(layer_module, th.nn.Conv2d) and layer_name != 'conv1':
                if layer_name.find('conv1') != -1:
                    in_channels = [i for i in range(layer_module.weight.shape[1])]
                    out_channels = indices[conv_layer]
                    layer_module.weight = th.nn.Parameter(
                        th.FloatTensor(
                            th.from_numpy(layer_module.weight.data.cpu().numpy()[out_channels])
                        ).to('cuda')
                    )

                if layer_name.find('conv2') != -1:
                    in_channels = indices[conv_layer]
                    out_channels = [i for i in range(layer_module.weight.shape[0])]
                    layer_module.weight = th.nn.Parameter(
                        th.FloatTensor(
                            th.from_numpy(layer_module.weight.data.cpu().numpy()[:, in_channels])
                        ).to('cuda')
                    )
                    conv_layer += 1

                layer_module.in_channels = len(in_channels)
                layer_module.out_channels = len(out_channels)

            if isinstance(layer_module, th.nn.BatchNorm2d) and layer_name != 'bn1' and layer_name.find('bn1') != -1:
                out_channels = indices[conv_layer]

                layer_module.weight = th.nn.Parameter(
                    th.FloatTensor(
                        th.from_numpy(layer_module.weight.data.cpu().numpy()[out_channels])
                    ).to('cuda')
                )
                layer_module.bias = th.nn.Parameter(
                    th.FloatTensor(
                        th.from_numpy(layer_module.bias.data.cpu().numpy()[out_channels])
                    ).to('cuda')
                )
                
                layer_module.running_mean = th.from_numpy(
                    layer_module.running_mean.cpu().numpy()[out_channels]
                ).to('cuda')
                layer_module.running_var = th.from_numpy(
                    layer_module.running_var.cpu().numpy()[out_channels]
                ).to('cuda')

                layer_module.num_features = len(out_channels)

            if isinstance(layer_module, nn.Linear):
                break

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)

class ResBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1):
        super(ResBasicBlock, self).__init__()
        self.inplanes = inplanes
        self.planes = planes
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.stride = stride
        self.shortcut = nn.Sequential()
        if stride != 1 or inplanes != planes:
            self.shortcut = LambdaLayer(
                lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes // 4, planes - inplanes - (planes // 4)), "constant", 0))

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(x)
        out = self.relu2(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, num_layers, covcfg, num_classes=10):
        super(ResNet, self).__init__()
        assert (num_layers - 2) % 6 == 0, 'depth should be 6n+2'
        n = (num_layers - 2) // 6
        self.covcfg = covcfg
        self.num_layers = num_layers

        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(1, block, 16, blocks=n, stride=1)
        self.layer2 = self._make_layer(2, block, 32, blocks=n, stride=2)
        self.layer3 = self._make_layer(3, block, 64, blocks=n, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

        self.linear = nn.Linear(64 * block.expansion, num_classes)

        self.initialize()

    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, a, block, planes, blocks, stride):
        layers = []
        layers.append(block(self.inplanes, planes, stride))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

def resnet_110():
    cov_cfg = [(3 * i + 2) for i in range(9 * 6 * 2 + 1)]
    return ResNet(ResBasicBlock, 110, cov_cfg)

# Load the model
device = th.device("cuda" if th.cuda.is_available() else "cpu")
model = resnet_110().to(device)

# Define optimizer and scheduler
optimizer = th.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = th.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 30], gamma=0.1)
criterion = nn.CrossEntropyLoss()

print("done")

Files already downloaded and verified
Files already downloaded and verified
done


In [4]:
criterion = nn.CrossEntropyLoss()

checkpoint = th.load('base.pth')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [5]:
train_acc = []
test_acc = []

with th.no_grad():
    
    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)
        output = model(inputs)
        y_hat = th.argmax(output, 1)
        train_acc.append((y_hat == targets).sum().item())
    
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        output = model(inputs)
        y_hat = th.argmax(output, 1)
        test_acc.append((y_hat == targets).sum().item())

train_acc = sum(train_acc) * 100 / len(trainloader.dataset)
test_acc = sum(test_acc) * 100 / len(testloader.dataset)

print(f'Train Accuracy: {train_acc:.2f}%')
print(f'Test Accuracy: {test_acc:.2f}%')

Train Accuracy: 97.63%
Test Accuracy: 90.30%
